In [ ]:
import geemap 
import ee
import os
# Authenticate to Earth Engine
ee.Authenticate()

# extents
extents = [
    {
        'name': 'bi',
        'geometry': ee.Geometry.Polygon([[[-154.668, 18.849],[-154.668, 20.334],[-156.243, 20.334],[-156.243, 18.849]]]),
    },
    {
        'name': 'ka',
        'geometry': ee.Geometry.Polygon([[[-159.2512, 21.819],[-159.2512, 22.269],[-159.816, 22.269],[-159.816, 21.819]]]),
    },
    {
        'name': 'mn',
        'geometry': ee.Geometry.Polygon([[[-155.9257, 20.343],[-155.9257, 21.32175],[-157.35, 21.32175],[-157.35, 20.343]]]),
    },
    {
        'name': 'oa',
        'geometry': ee.Geometry.Polygon([[[-157.602, 21.18],[-157.602, 21.7425],[-158.322, 21.7425],[-158.322, 21.18]]]),
    }
]

# Loop through each extent
for extent in extents:
    # Create a Map for visualization
    Map = geemap.Map()
    Map.centerObject(extent['geometry'])
    Map.addLayer(extent['geometry'], {'color': 'red'}, extent['name'])

    # Load MODIS data
    modis = ee.ImageCollection('MODIS/061/MOD09GA')

    # Define start and end years
    start_year = 2023  
    end_year = 2024   

    # Define the interval in days
    interval = 16

    # Loop over the years
    for year in range(start_year, end_year + 1):
        # Define the start and end dates for the current year
        start_date = ee.Date.fromYMD(year, 1, 1)
        end_date = ee.Date.fromYMD(year, 12, 31)

        # Generate a list of dates with the specified interval
        dates = ee.List.sequence(start_date.millis(), end_date.advance(1, 'day').millis(), interval * 24 * 60 * 60 * 1000)

        # Loop over the dates
        date_list = dates.getInfo()
        for dateMillis in date_list:
            date = ee.Date(dateMillis)

            # Filter the collection for the current date and extent
            filtered = modis.filterDate(date, date.advance(interval, 'day')).filterBounds(extent['geometry'])

            # Cloud masking function
            def maskMODISclouds(image):
                qa = image.select('state_1km')
                cloudBitMask = 1 << 10
                mask = qa.bitwiseAnd(cloudBitMask).eq(0)
                return image.updateMask(mask)

            filtered1 = filtered.map(maskMODISclouds)

            # NDVI calculation function
            def addNDVI(image):
                ndvi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('ndvi')
                return image.addBands(ndvi)

            withNdvi = filtered1.map(addNDVI)

            # Gap-filling function
            def fill(image):
                ima = image.focal_mean(1, 'square', 'pixels', 20)
                return ima.blend(image)

            filled = withNdvi.map(fill)

            # Clip NDVI based on the extent
            NDVIsen = filled.median().clip(extent['geometry'])

            # Create a folder based on the extent's name
            folder_name = extent['name']
            out_dir = os.path.expanduser(f'C:/Users/Sayed/Desktop/Hawaii NDVI/ndviex/{folder_name}')
            
            # Ensure the folder exists or create it if not
            os.makedirs(out_dir, exist_ok=True)

            # Export the image to the folder with the extent names and dates
            file_name = f'{extent["name"]}_NDVI_{year}_{date.format("YYYYMMdd").getInfo()}.tif'
            file_path = os.path.join(out_dir, file_name)

            geemap.ee_export_image(NDVIsen.select('ndvi'), filename=file_path, scale=250, region=extent['geometry'])
